## Testing Category Discovery Using KCM

In [115]:
%load_ext autoreload
%autoreload 2

from sklearn.svm import SVC
from kcm.koopman_category_model import KoopmanCategoryModel
from kcm.discovery import sup_con_loss, BaselineModel, HASHHead, cluster_acc, split_cluster_acc_v1, split_cluster_acc_v2, create_hash_ids
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, adjusted_mutual_info_score

import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [251]:
num_cats = 4
num_samples = 100
system_dimension = 1
delay_embeddings = 3
num_segments = 10
svd_rank = None
dmd_rank = None
q = 2
num_clusters = 7
test_size = 0.2
codebook_training_size = 300 # divides <num training classes> * num_segments 
category_discovery=True
train_classes = [0,1,2]
seed = 42

KCM = KoopmanCategoryModel(num_cats=num_cats,
                           num_samples=num_samples,
                           system_dimension=system_dimension,
                           delay_embeddings=delay_embeddings,
                           num_segments=num_segments,
                           svd_rank=svd_rank,
                           dmd_rank=dmd_rank,
                           q=q,
                           cluster_method='kmeans',
                           num_clusters=num_clusters,
                           seed=seed)

KCM.generate_data()
KCM.train_test_split(test_size=test_size,
                     codebook_training_size=codebook_training_size,
                     category_discovery=category_discovery,
                     train_classes=train_classes)
KCM.create_codebook(include_plots=False)

Loading data in at C:\Users\dbpet\Documents\JHU\EN.625.801-802\koopman-category-discovery\data\1-dimensional-systems\dataset_4_class_100_noisy_samples.pkl...


(1/2) * 300^2 = 45000 Wasserstein distance metrics: 100%|█████████████████████████████████████████| 300/300 [12:51<00:00,  2.57s/it]
7 * 1600 = 11200 Wasserstein distance metrics: 100%|██████████████████████████████████████████████████| 7/7 [02:33<00:00, 21.90s/it]


In [253]:
KCM.save()
KCM.shutdown_logger()

PicklingError: Can't pickle <class 'kcm.koopman_category_model.KoopmanCategoryModel'>: it's not the same object as kcm.koopman_category_model.KoopmanCategoryModel

In [254]:
X_train = KCM.inv_c_train_matrix
y_train = KCM.train_target

X_test = KCM.inv_c_test_matrix
y_test = KCM.test_target

X_train = torch.Tensor(X_train)
y_train = torch.tensor(y_train).long().squeeze()

X_test = torch.Tensor(X_test)
y_test = torch.tensor(y_test).long().squeeze()

In [255]:
y_train.unique() # 80, 0, 80, 80 = 240 training samples (5 segments)

tensor([0, 1, 2])

In [256]:
y_test.unique() # 20, 100, 20, 20 = 160 testing samples (5 segments)

tensor([0, 1, 2, 3])

In [257]:
unique_histograms, indices = torch.unique(X_train, dim=0, return_inverse=True)

print('Histograms with Duplicate Labels:\n')
num_issue_hists = 0
num_issue_samples = 0
for uni, ind in zip(unique_histograms,torch.unique(indices)):

    histogram_count = X_train[indices == ind].shape[0]
    unique_labels = torch.unique(y_train[indices == ind])

    if len(unique_labels) > 1:
        num_issue_hists += 1
        num_issue_samples += histogram_count
        print(f'{np.round(np.array(uni),3)} has {histogram_count} instances with {np.array(unique_labels)} unique_labels')


percent_hist_issues = round(num_issue_hists*100/len(unique_histograms),2)
print(f'\n{num_issue_hists}/{len(unique_histograms)} ({percent_hist_issues} %) of histograms had duplicate labels,')
perc_sample_issues = round(num_issue_samples*100/(KCM.num_cats*KCM.num_samples),2)
print(f'\taffecting {num_issue_samples}/{KCM.num_cats*KCM.num_samples} samples ({perc_sample_issues} %)')

Histograms with Duplicate Labels:

[0.    0.096 0.    0.    0.229 0.    0.   ] has 2 instances with [1 2] unique_labels
[0.    0.161 0.    0.    0.164 0.    0.   ] has 13 instances with [1 2] unique_labels
[0.    0.193 0.    0.    0.131 0.    0.   ] has 13 instances with [1 2] unique_labels
[0.    0.225 0.    0.    0.098 0.    0.   ] has 20 instances with [1 2] unique_labels
[0.    0.257 0.    0.    0.065 0.    0.   ] has 10 instances with [1 2] unique_labels
[0.    0.289 0.    0.    0.033 0.    0.   ] has 5 instances with [1 2] unique_labels
[0.118 0.096 0.    0.    0.196 0.    0.   ] has 2 instances with [1 2] unique_labels
[0.118 0.161 0.    0.    0.131 0.    0.   ] has 6 instances with [1 2] unique_labels

8/127 (6.3 %) of histograms had duplicate labels,
	affecting 71/400 samples (17.75 %)


In [ ]:
input_dim = X_train.shape[1]
output_dim = 8
hidden_dims = [200, 200] # [1024, 512, 256]
dropout = 0.3
classes = KCM.num_cats
test_size = 0.25
epochs = 1000
model_type = 'SMILE' # baseline, SMILE

if model_type == 'baseline':
    model = BaselineModel(input_dim, output_dim, hidden_dims, dropout)
elif model_type == 'SMILE':
    model = HASHHead(input_dim, output_dim, hidden_dims, dropout)


optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


true_y = KCM.df_test[['target','sample']].drop_duplicates()['target'].values
mask = np.isin(true_y,KCM.train_classes)


model.train()

train_aris = []
train_nmis = []
train_amis = []

test_aris = []
test_nmis = []
test_amis = []

weighted_scores = []
old_scores = []
new_scores = []

global_scores = []
old_global_scores = []
new_global_scores = []

unique_training_hashes = []
unique_testing_hashes = []

losses = []
for i in tqdm(range(epochs), desc='Training Model'):

    if model_type == 'baseline':
        training_features, _, _ = model(X_train)
        testing_features, _, _ = model(X_test)
        sc_loss = sup_con_loss(training_features, y_train, temp=0.2)
        reg_loss = 0
        
    elif model_type == 'SMILE':
        training_features, training_hash_features, _ = model(X_train)
        testing_features, testing_hash_features, _ = model(X_test)
        sc_loss = sup_con_loss(training_features, y_train, temp=0.2)

        reg_loss = (1 - torch.abs(training_hash_features)).mean()
        

    loss = sc_loss * 1 + reg_loss * 3
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    losses.append(loss.item())

    # Create Cluster ids for train, test, and full data sets
    training_hash_ids, training_hashes, new_training_labels = create_hash_ids(training_features, y_train)
    testing_hash_ids, testing_hashes, new_testing_labels = create_hash_ids(testing_features, y_test)
    
    # # Full dataset clusters
    # features, _, _ = model(X)
    # full_has_ids, full_hashes, new_full_labels = create_hash_ids(features, y)
    
    # Report ARI, NMI, and AMI for training and testing splits
    train_aris.append(adjusted_rand_score(y_train, training_hash_ids))
    train_nmis.append(normalized_mutual_info_score(y_train, training_hash_ids))
    train_amis.append(adjusted_mutual_info_score(y_train, training_hash_ids))

    test_aris.append(adjusted_rand_score(y_test, testing_hash_ids))
    test_nmis.append(normalized_mutual_info_score(y_test, testing_hash_ids))
    test_amis.append(adjusted_mutual_info_score(y_test, testing_hash_ids))

    # Report category discovery metrics
    total_acc, old_acc, new_acc = split_cluster_acc_v1(np.array(y_test), testing_hash_ids, mask)
    weighted_scores.append(total_acc)
    old_scores.append(old_acc)
    new_scores.append(new_acc)
    
    total_acc, old_acc, new_acc = split_cluster_acc_v2(np.array(y_test), testing_hash_ids, mask)
    global_scores.append(total_acc)
    old_global_scores.append(old_acc)
    new_global_scores.append(new_acc)

    unique_training_hashes.append(len(np.unique(training_hash_ids)))
    unique_testing_hashes.append(len(np.unique(testing_hash_ids)))

Training Model:  90%|████████████████████████████████████████████████████████████████████▎       | 899/1000 [01:00<00:06, 15.05it/s]

In [ ]:
plt.figure(figsize=(5,2))
plt.plot(np.log(np.array(losses)))
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Supervised Contrastive Loss')
plt.show()

In [ ]:
plt.figure(figsize=(5,2))
plt.plot(unique_training_hashes,label='Training')
plt.plot(unique_testing_hashes,label='Testing')
plt.xlabel('Epoch')
plt.ylabel('Unique Hashes')
plt.title('Number of Unique Hashes')
plt.legend()
plt.show()

In [ ]:
figs, axes = plt.subplots(1,3,figsize=(12,3))

minimum = -0.05
maximum = 1.05

# Plot independent & weighted scores
axes[0].plot(old_scores,linewidth=0.8,color='blue',label='Old Classes')
axes[0].plot(new_scores,linewidth=0.8,color='red',label='New Classes')
axes[0].plot(weighted_scores,linewidth=2,color='purple',label='Total')
axes[0].set_title('Independent Scores')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Score')
axes[0].set_ylim([minimum, maximum])
axes[0].legend()

# Plot global scores
axes[1].plot(old_global_scores,linewidth=0.8,color='blue',label='Old Classes')
axes[1].plot(new_global_scores,linewidth=0.8,color='red',label='New Classes')
axes[1].plot(global_scores,linewidth=2,color='purple',label='Total')
axes[1].set_title('Global Scores')
axes[1].set_xlabel('Epoch')
axes[1].set_ylim([minimum, maximum])
axes[1].legend()

# Plot training and testing aris
axes[2].plot(train_aris,linewidth=2,color='lightblue',label='Train ARI')
axes[2].plot(train_nmis,linewidth=1,color='mediumblue',label='Train NMI')
axes[2].plot(train_amis,linewidth=0.5,color='darkblue',label='Train AMI')
axes[2].plot(test_aris,linewidth=2,color='lightcoral',label='Test ARI')
axes[2].plot(test_nmis,linewidth=1,color='red',label='Test NMI')
axes[2].plot(test_amis,linewidth=0.5,color='darkred',label='Test AMI')
axes[2].set_title('')
axes[2].set_xlabel('Epoch')
axes[2].set_ylim([minimum, maximum])
axes[2].legend()


plt.suptitle('Category Discovery Metrics')
plt.tight_layout()
plt.show()

In [ ]:
# Calculate basic statistics (means, variances, angular momentums, periodicity, etc.) - to construct a feature vector for comparison

# Try introducing noise to see how koopman feature extractor compares with other methods - statistical methods will suffer big time with noise
# Comparing with random features for comparison to random

# Play around with the size of the encoder (more parameters than features?)
# Try making num_parameters < num_data inputs (maybe ~20%) - make only 2 layers with smaller hidden dimensions

# Change size of hash vector

# How to augment the dataset for optimized category discovery - add more categories
# Can training on more classes help discover more categories?
# Test out changing training and testing class sizes
# Compare train on 1-3 and discover 3-1

# Send some basic figures to Cetin and Charlie


# Observations

# With a smaller neural network, it just bunches a lot of the samples into the same categories, particulary the non-harmonic ones
# The network has higher accuracy at the very beginning - this could be due to the repetitive nature of the histograms from KCM
# There are some histogram values that are repeated across different systems

In [ ]:
def plot_histograms(df,combine=True,y_to_color=None,all_y_vals=None):

    all_clusters = sorted(df['cluster_id'].unique())
    unique_y = sorted(df['y'].unique()) if all_y_vals is None else sorted(all_y_vals)
    
    x = np.arange(len(all_clusters))
    width = 0.8 / len(unique_y)

    if y_to_color is None:
        cmap = plt.get_cmap('tab10')
        y_to_color = {y: cmap(i % 10) for i, y in enumerate(unique_y)}
        

    if combine:
        plt.figure(figsize=(8,3))
        for i, y_val in enumerate(unique_y):
            counts = df.loc[df['y'].eq(y_val),'cluster_id'].value_counts().reindex(all_clusters, fill_value=0)
            pmf = counts / counts.sum()
            bar_positions = x + i * width
            plt.bar(bar_positions, pmf, width=width, label=f'y = {y_val}', color=y_to_color.get(y_val, 'gray'))
        
        plt.xlabel('cluster_id')
        plt.ylabel('Frequency')
        plt.title('Cluster ID Value Counts by y')
        plt.xticks(x + width * (len(unique_y) - 1) / 2, labels=all_clusters)
        plt.legend(title='y value')
        plt.show()

    
    else:
        fig, axs = plt.subplots(len(unique_y), 1, figsize=(6, 10), constrained_layout=True)
        for i, y_val in enumerate(unique_y):
            counts = train.loc[train['y'].eq(y_val),'cluster_id'].value_counts().reindex(all_clusters, fill_value=0)
            pmf = counts / counts.sum()
            axs[i].bar(all_clusters, pmf, color=y_to_color.get(y_val, 'gray'))
            axs[i].set_title(f'Histogram of cluster_ids for y = {y_val}')
            axs[i].set_xlabel('cluster_id')
            axs[i].set_ylabel('Frequency')
            axs[i].set_xticks(all_clusters)
        plt.show()

In [ ]:
# data = {'cluster_id' : training_cluster_ids, 'y' : y_train}
data = {'cluster_id' : training_hash_ids, 'y' : y_train}
train = pd.DataFrame(data)

# data = {'cluster_id' : testing_cluster_ids, 'y' : y_test}
data = {'cluster_id' : testing_hash_ids, 'y' : y_test}
test = pd.DataFrame(data)

for i in range(output_dim):
    train[f'hash_{i}'] = training_hashes[:,i]
    test[f'hash_{i}'] = testing_hashes[:,i]

all_y_vals = sorted(set(train['y']).union(set(test['y'])))

# Create consistent color mapping
cmap = plt.get_cmap('tab10')
y_to_color = {y: cmap(i % 10) for i, y in enumerate(all_y_vals)}


plot_histograms(train,combine=True,y_to_color=y_to_color)
plot_histograms(test,combine=True,y_to_color=y_to_color)

In [ ]:
# [0,1,0,1,0,1,0,1]
# [1,1,1,1,0,0,0,0]